## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-17-11-49-44 +0000,nyt,Ukraine Weighs Trump’s Offer of Security Guara...,https://www.nytimes.com/2025/08/17/world/europ...
1,2025-08-17-11-44-35 +0000,bbc,Air Canada flights to resume after union told ...,https://www.bbc.com/news/articles/cwyex8489gno...
2,2025-08-17-11-42-52 +0000,wapo,European leaders to join Trump-Zelensky meetin...,https://www.washingtonpost.com/world/2025/08/1...
3,2025-08-17-11-40-00 +0000,wapo,The Ukrainian children killed in Russian strik...,https://www.washingtonpost.com/world/interacti...
4,2025-08-17-11-36-18 +0000,nypost,Minnesota fast food worker Rodolfo Depaz who w...,https://nypost.com/2025/08/17/us-news/minnesot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,38
269,putin,24
0,ukraine,18
14,strike,13
120,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...,117
102,2025-08-16-21-22-09 +0000,nyt,Trump Backs Off Cease-Fire Demand in Ukraine W...,https://www.nytimes.com/2025/08/16/world/europ...,114
132,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...,110
127,2025-08-16-19-25-47 +0000,wapo,Russia sees victory as Trump adopts Putin’s ap...,https://www.washingtonpost.com/world/2025/08/1...,107
98,2025-08-16-21-40-53 +0000,wapo,"Trump drops ceasefire demand for Ukraine war, ...",https://www.washingtonpost.com/world/2025/08/1...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,117,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...
112,53,2025-08-16-20-15-27 +0000,wapo,Canada orders Air Canada flight attendants bac...,https://www.washingtonpost.com/world/2025/08/1...
121,48,2025-08-16-19-41-53 +0000,nypost,Trump’s efforts at Alaska summit applauded by ...,https://nypost.com/2025/08/16/world-news/trump...
143,39,2025-08-16-16-37-41 +0000,nypost,Zelensky warns Putin will step up attacks on U...,https://nypost.com/2025/08/16/world-news/zelen...
18,31,2025-08-17-10-00-00 +0000,latimes,News Analysis: Newsom's decision to fight fire...,https://www.latimes.com/california/story/2025-...
126,27,2025-08-16-19-28-00 +0000,wsj,More National Guard troops are heading soon to...,https://www.wsj.com/politics/national-security...
4,26,2025-08-17-11-36-18 +0000,nypost,Minnesota fast food worker Rodolfo Depaz who w...,https://nypost.com/2025/08/17/us-news/minnesot...
36,24,2025-08-17-08-57-49 +0000,bbc,Palestinians flee IDF attacks on Gaza City as ...,https://www.bbc.com/news/articles/c2018wx3zlgo...
93,24,2025-08-16-22-11-19 +0000,nypost,BB gun-wielding man fatally shot by off-duty c...,https://nypost.com/2025/08/16/us-news/bb-gun-w...
53,23,2025-08-17-04-05-37 +0000,nypost,Florida woman arrested after making drunken bo...,https://nypost.com/2025/08/17/us-news/florida-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
